In [2]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 6.4 MB 20.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 1.9 MB/s 


In [3]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols


#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
def get_crypto_syms():
   screens = ['all_cryptocurrencies_au',
 'all_cryptocurrencies_ca',
 'all_cryptocurrencies_eu',
 'all_cryptocurrencies_gb',
 'all_cryptocurrencies_in',
 'all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def process(dfs): 
   fixed_dfs = []
   for df in dfs:
       fixed_df = []
       df = np.array(df)
       for i in range(20, df.shape[0]-1):
           twenty_days_ago = i-20
           ninteen_days_ago = i-19
           eighteen_days_ago = i-18
           seventeen_days_ago = i-17
           sixteen_days_ago = i-16
           fifteen_days_ago = i-15
           fourteen_days_ago = i-14
           thirteen_days_ago = i-13
           twelve_days_ago = i-12
           eleven_days_ago = i-11
           ten_days_ago = i-10
           nine_days_ago = i-9
           eight_days_ago = i-8
           seven_days_ago = i-7
           six_days_ago = i-6
           five_days_ago = i-5
           four_days_ago = i-4
           three_days_ago = i-3
           two_days_ago = i - 2
           yesterday = i - 1
           today = i
           tomorrow = i + 1
           if df[tomorrow][3] > df[today][3]:
               future = 1
           else:
               future = 0


           row = [df[twenty_days_ago][3],df[ninteen_days_ago][3],df[eighteen_days_ago][3],df[seventeen_days_ago][3],
                  df[sixteen_days_ago][3],df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],
                  df[twelve_days_ago][3],df[eleven_days_ago][3],df[ten_days_ago][3], df[nine_days_ago][3], 
                  df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3], df[five_days_ago][3], 
                  df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                  df[today][3], future]

           fixed_df.append(row)


       arrayed_fixed_df = np.array([fixed_df])
       two_d_fixed_df = []
       for i in range(0, arrayed_fixed_df.shape[1]):
          two_d_fixed_df.append(arrayed_fixed_df[0][i])
    
       mm = np.array(two_d_fixed_df)


       column_names = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago",
                       "seventeen_days_ago","sixteen_days_ago","fifteen_days_ago",
                       "fourteen_days_ago","thirteen_days_ago","twelve_days_ago",
                       "eleven_days_ago","ten_days_ago", "nine_days_ago", 
                       "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago",
                       "two_days_ago", "yesterday","today", "result"]
    
       column_names_without_result = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago",
                                      "seventeen_days_ago","sixteen_days_ago","fifteen_days_ago",
                                      "fourteen_days_ago","thirteen_days_ago","twelve_days_ago",
                                      "eleven_days_ago",   "ten_days_ago", "nine_days_ago", 
                                      "eight_days_ago", "seven_days_ago", "six_days_ago",
                                      "five_days_ago", "four_days_ago", "three_days_ago", 
                                      "two_days_ago", "yesterday","today"]
       dff = pd.DataFrame(mm, columns=column_names)
       scaler = MinMaxScaler()
       result = dff["result"]
       dff = dff.drop(["result"],axis=1)
       scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=column_names_without_result,dtype=object)

       scaled["result"] = result
       
       fixed_dfs.append(scaled)

   data = fixed_dfs[0]
   for i in range(1,len(dfs)):
     data = pd.concat([data,fixed_dfs[i]], ignore_index = True)
   data = data.astype(float)
   data = data.dropna()
   ttext = getsizeof(data)
   #data.to_parquet(f"{name}.parquet")
   return data
   
def spliting(data):
  X = data.drop(["result"],axis=1)
  y = data["result"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
#symbols = get_crypto_syms()
#dfs = []
#for symbol in symbols:
#           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
#           if data.empty :
#             print("Passing...")
#           else:
#               dfs.append(data)
#ndfs = dfs
#dfs = []
#for df in ndfs:
#        if df.shape[0] > 21:
#           dfs.append(df)
#data = process(dfs)
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")
#xTrain, xTest, yTrain, yTest = spliting(data)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_parquet("/content/drive/MyDrive/Colab Files/forex_1d_scaled_yf.parquet")
xTrain, xTest, yTrain, yTest = spliting(data)

data

Mounted at /content/drive
(10267413, 21) (10267413,)
(1140824, 21) (1140824,)


,twenty_days_ago,ninteen_days_ago,eighteen_days_ago,seventeen_days_ago,sixteen_days_ago,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,...,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,0.419335,0.389203,0.573133,0.672316,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,...,0.271814,0.060264,0.000000,0.128688,0.337728,0.269303,0.218456,0.247332,0.148776,1.0
1,0.389203,0.573133,0.672316,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,0.246077,...,0.060264,0.000000,0.128688,0.337728,0.269303,0.218456,0.247332,0.148776,0.202762,1.0
2,0.573133,0.672316,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,0.246077,0.311362,...,0.000000,0.128688,0.337728,0.269303,0.218456,0.247332,0.148776,0.202762,0.263026,0.0
3,0.672316,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,0.246077,0.311362,0.271814,...,0.128688,0.337728,0.269303,0.218456,0.247332,0.148776,0.202762,0.263026,0.162586,1.0
4,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,0.246077,0.311362,0.271814,0.060264,...,0.337728,0.269303,0.218456,0.247332,0.148776,0.202762,0.263026,0.162586,0.317640,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11408232,0.452380,0.476190,0.690476,0.761904,0.476190,0.023808,0.000000,0.285714,0.285714,0.452380,...,0.571428,0.571428,0.785714,0.976190,0.952380,0.880952,0.928570,0.857142,1.000000,0.0
11408233,0.476190,0.690476,0.761904,0.476190,0.023808,0.000000,0.285714,0.285714,0.452380,0.476190,...,0.571428,0.785714,0.976190,0.952380,0.880952,0.928570,0.857142,1.000000,0.809522,0.0
11408234,0.690476,0.761904,0.476190,0.023808,0.000000,0.285714,0.285714,0.452380,0.476190,0.642856,...,0.785714,0.976190,0.952380,0.880952,0.928570,0.857142,1.000000,0.809522,0.714284,0.0
11408235,0.761904,0.476190,0.023808,0.000000,0.285714,0.285714,0.452380,0.476190,0.642856,0.571428,...,0.976190,0.952380,0.880952,0.928570,0.857142,1.000000,0.809522,0.714284,0.452380,0.0


In [5]:
model = Sequential()

model.add(Dense(5000, activation='relu', kernel_initializer='he_normal', input_shape=(xTrain.shape[1],)))
model.add(Dense(4000, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(3500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(3000, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(2000, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5000)              110000    
                                                                 
 dense_1 (Dense)             (None, 4000)              20004000  
                                                                 
 dense_2 (Dense)             (None, 3500)              14003500  
                                                                 
 dense_3 (Dense)             (None, 3000)              10503000  
                                                                 
 dense_4 (Dense)             (None, 2000)              6002000   
                                                                 
 dense_5 (Dense)             (None, 1500)              3001500   
                                                                 
 dense_6 (Dense)             (None, 1)                 1

In [ ]:
model.fit(xTrain,yTrain,epochs=15,batch_size=256,validation_data=(xTest,yTest))

Epoch 1/15


In [20]:
model.evaluate(xTest,yTest)

 240/3327 [=>............................] - ETA: 5s - loss: 0.9046 - accuracy: 0.4783

KeyboardInterrupt: ignored

In [29]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

KeyboardInterrupt: ignored